#Второй этап обучения AlphaGo.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#забираем из Google Drive если есть)))
!cp ./drive/MyDrive/colab1/dlgo.zip ./dlgo.zip

In [3]:
#распаковка модулей DL-Go
!unzip -o dlgo.zip

Archive:  dlgo.zip
   creating: dlgo/
   creating: dlgo/agent/
  inflating: dlgo/agent/base.py      
  inflating: dlgo/agent/helpers.py   
  inflating: dlgo/agent/naive.py     
  inflating: dlgo/agent/pg.py        
  inflating: dlgo/agent/predict.py   
  inflating: dlgo/agent/termination.py  
 extracting: dlgo/agent/__init__.py  
   creating: dlgo/agent/__pycache__/
  inflating: dlgo/agent/__pycache__/base.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/base.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/helpers.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/naive.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/predict.cpython-36.pyc  
  inflating: dlgo/agent/__pycache__/termination.cpython-310.pyc  
  inflating: dlgo/agent/__pycache__/termination.cpython-36.pyc  
  inflating: dlgo/

In [4]:
!cp ./drive/MyDrive/colab1/alphago_sl_policy.h5 ./alphago_sl_policy.h5

##Загрузка двух версий обученной сети политики для создания двух противников

In [5]:
from dlgo.agent.pg import PolicyAgent
from dlgo.agent.predict import load_prediction_agent
from dlgo.encoders.alphago import AlphaGoEncoder
from dlgo.rl.simulate import experience_simulation
import h5py

In [6]:
encoder = AlphaGoEncoder()

In [7]:
sl_agent = load_prediction_agent(h5py.File('alphago_sl_policy.h5'))
sl_opponent = load_prediction_agent(h5py.File('alphago_sl_policy.h5'))

In [8]:
alphago_rl_agent = PolicyAgent(sl_agent.model, encoder)
opponent = PolicyAgent(sl_opponent.model, encoder)

##Генерация данных игры агента с самим собой для обучения агента PolicyAgent

In [9]:
%%time
#num_games = 1000
#num_games = 45 --> память закончилась 12.68GB
num_games = 30
experience = experience_simulation(num_games, alphago_rl_agent, opponent)

Выходные данные были обрезаны до нескольких последних строк (5000).
1/1 [==============================] - 0s 21ms/step
Победил: Player.white
Игра 21/30
1/1 [==============================] - 0s 17ms/step
Победил: Player.white
Игра 22/30
1/1 [==============================] - 0s 67ms/step
Победил: Player.white
Игра 23/30
1/1 [==============================] - 0s 23ms/step
Победил: Player.white
Игра 24/30
1/1 [==============================] - 0s 30ms/step
Победил: Player.white
Игра 25/30
1/1 [==============================] - 0s 31ms/step
Победил: Player.black
Игра 26/30
1/1 [==============================] - 0s 36ms/step
Победил: Player.white
Игра 27/30
1/1 [==============================] - 0s 24ms/step
Победил: Player.black
Игра 28/30
1/1 [==============================] - 0s 43ms/step
Победил: Player.white
Игра 29/30
1/1 [==============================] - 0s 31ms/step
Победил: Player.white
Игра 30/30
1/1 [==============================] - 0s 32ms/step
Победил: Player.black
CPU time

In [10]:
alphago_rl_agent.train(experience)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


28/28 [==============================] - 22s 584ms/step - loss: -0.0259


In [11]:
with h5py.File('alphago_rl_policy.h5', 'w') as rl_agent_out:
    alphago_rl_agent.serialize(rl_agent_out)

In [12]:
with h5py.File('alphago_rl_experience.h5', 'w') as exp_out:
    experience.serialize(exp_out)

In [13]:
!cp ./alphago_rl_policy.h5 ./drive/MyDrive/colab1/alphago_rl_policy.h5
!cp ./alphago_rl_experience.h5 ./drive/MyDrive/colab1/alphago_rl_experience.h5